In [ ]:
import os
from copy import deepcopy

import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats
import scipy.ndimage
from tqdm import tqdm_notebook as tqdm

import distributed

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
import matplotlib.dates as mdates
import matplotlib.lines as mpll

In [ ]:
rnd = np.random.RandomState(42)

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')
plt.rcParams['text.usetex'] = False
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
cluster = distributed.LocalCluster(dashboard_address=':8787')
client = distributed.Client(cluster, )
client

# Load data

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/da_enkf_for_soil'

## H2O

In [ ]:
vr_h2o_path = os.path.join(base_path, '016', 'h2o_cleaned.nc')
vr_h2o = xr.open_dataset(vr_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
vr_fluxes_path = os.path.join(base_path, '016', 'fluxes.nc')
vr_fluxes = xr.open_dataset(vr_fluxes_path).squeeze(drop=True).chunk({'time':1, 'lat':302, 'lon':267}).isel(lat=slice(30, -30), lon=slice(30, -30))

In [ ]:
da_h2o_path = os.path.join(base_path, '020', 'h2o_cleaned.nc')
da_h2o = xr.open_dataset(da_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((40, 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_fluxes_path = os.path.join(base_path, '020', 'fluxes.nc')
da_fluxes = xr.open_dataset(da_fluxes_path).squeeze(drop=True).chunk({'ensemble':40, 'time':1, 'lat':302, 'lon':267}).isel(lat=slice(30, -30), lon=slice(30, -30))

## Additional data

In [ ]:
const_path = '/work/um0203/u300636/for2131/runs/utilities/clm_aux.nc'
const_data = xr.open_dataset(const_path).isel(time=0)
sat_point = const_data['WATSAT'].isel(levsoi=4)[..., 30:-30, 30:-30]

## Pre-process data

In [ ]:
da_bg_fluxes = da_fluxes.isel(time=~da_fluxes.indexes['time'].duplicated())
da_bg_h2o = da_h2o.isel(time=~da_h2o.indexes['time'].duplicated())
vr_h2o = vr_h2o.isel(time=~vr_h2o.indexes['time'].duplicated())

# Estimate fluxes

In [ ]:
da_latent = da_fluxes['FCEV'] + da_fluxes['FCTR'] + da_fluxes['FGEV']
da_sensible = da_fluxes['FSH']

vr_latent = vr_fluxes['FCEV'] + vr_fluxes['FCTR'] + vr_fluxes['FGEV']
vr_sensible = vr_fluxes['FSH']

## Get sensible heat flux in dependence on the SMI

In [ ]:
da_sensible = da_sensible[:, 3::4][:, 6:].load()
da_smi = (da_h2o / sat_point)[:, 3::4][:, 6:].load()

In [ ]:
da_hours = da_smi.copy(deep=True)
da_hours[:] = da_hours.indexes['time'].hour.values[None, :, None, None]

In [ ]:
smi_bins = np.arange(-0.025, 1, 0.05)
center_bins = smi_bins[:-1] + 0.025
tmp_stats = scipy.stats.binned_statistic(
    da_smi.isel(time=da_smi.indexes['time'].hour == 14).values.flatten(),
    da_sensible.isel(time=da_smi.indexes['time'].hour == 14).values.flatten(),
    bins=smi_bins, statistic='median'
)[0]

In [ ]:
tmp_10 = scipy.stats.binned_statistic(
    da_smi.isel(time=da_smi.indexes['time'].hour == 14).values.flatten(),
    da_sensible.isel(time=da_smi.indexes['time'].hour == 14).values.flatten(),
    bins=smi_bins, statistic=lambda x: np.percentile(x, 5)
)[0]

In [ ]:
tmp_90 = scipy.stats.binned_statistic(
    da_smi.isel(time=da_smi.indexes['time'].hour == 14).values.flatten(),
    da_sensible.isel(time=da_smi.indexes['time'].hour == 14).values.flatten(),
    bins=smi_bins, statistic=lambda x: np.percentile(x, 95)
)[0]

In [ ]:
cmap = plt.get_cmap('Reds')
cmap_colors = cmap(np.arange(cmap.N))
cmap = mplc.ListedColormap(cmap_colors[:-96])
cmap.set_under(color='white')
norm = mplc.LogNorm(vmin=10, vmax=2E4)

figsize = [s*0.48 for s in plt.rcParams['figure.figsize']]

fig, ax = plt.subplots(figsize=figsize)
#plt_hexbin = ax.hexbin(
#    da_smi.isel(time=da_sensible.indexes['time'].hour == 14).values.flatten(),
#    da_sensible.isel(time=da_sensible.indexes['time'].hour == 14).values.flatten(),
#    cmap=cmap, norm=norm
#)
ax.axhline(c='black', lw=0.5)
tmp_line = ax.plot(center_bins, tmp_stats, c='black', ls='-', alpha=1.0)
perc_line = ax.plot(center_bins, tmp_10, c='0.2', ls='--', alpha=1.0)
_ = ax.plot(center_bins, tmp_90, c='0.2', ls='--', alpha=1.0)

ax.axvline(0.2, c='black', alpha=0.5, lw=0.5)
ax.axvline(0.5, c='black', alpha=0.5, lw=0.5)
ax.text(0.1, -50, s='dry', ha='center', va='center')
ax.text(0.35, -50, s='mixed', ha='center', va='center')
ax.text(0.675, -50, s='moist', ha='center', va='center')
ax.set_ylabel('Sensible heat flux (W/m$^2$)')
ax.set_xlabel('Soil moisture saturation')
ax.set_xlim(0, 0.85)
ax.set_xticks(np.arange(0, 1, 0.2))
ax.set_ylim(-100, 500)

#cbar = fig.colorbar(plt_hexbin)
#cbar.set_label('Number of points within bin')

legend = ax.legend([tmp_line[0], perc_line[0]], ['Median', '5. / 95. Percentile'], loc=4, bbox_to_anchor=(1.04, 0.95), ncol=2)

fig.savefig('../figures/fig_08_sensible_sat.png', dpi=300)
plt.show()